In [ ]:
# stock_update.py
import streamlit as st
import pandas as pd
import os
from utils.file_handler import read_po_file

DATA_PATH = "data/warehouse_stock.csv"
os.makedirs("data", exist_ok=True)

def run():
    st.title("🚀 Stock Update & Sync")
    st.caption("Upload new PO sheets or manually adjust stock levels in real-time.")

    uploaded_file = st.file_uploader("📤 Upload PO File (CSV, Excel, PDF)", type=["csv", "xlsx", "pdf"])

    if uploaded_file:
        df, _ = read_po_file(uploaded_file)
        if not df.empty:
            st.success(f"✅ Loaded {len(df)} items from PO sheet.")
            st.dataframe(df.head(10))
            if st.button("📥 Sync to Main Database"):
                sync_stock(df)
                st.success("Database successfully updated.")
        else:
            st.warning("⚠️ Could not extract data. Please check file format.")

    st.markdown("### ✏️ Manual Adjustments")
    if os.path.exists(DATA_PATH):
        stock_df = pd.read_csv(DATA_PATH)
        edited_df = st.data_editor(stock_df, num_rows="dynamic", use_container_width=True)
        if st.button("💾 Save Manual Changes"):
            edited_df.to_csv(DATA_PATH, index=False)
            st.success("✅ Stock data saved successfully!")
    else:
        st.warning("No stock data found to edit. Please upload one first.")

def sync_stock(new_data):
    """Merge new PO data with existing warehouse stock."""
    if os.path.exists(DATA_PATH):
        current_df = pd.read_csv(DATA_PATH)
        merged = pd.concat([current_df, new_data], ignore_index=True)
        merged.drop_duplicates(subset=["Sku", "Location"], keep="last", inplace=True)
    else:
        merged = new_data
    merged.to_csv(DATA_PATH, index=False)